In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools

from scipy import stats
from statsmodels.stats.descriptivestats import sign_test
from statsmodels.stats.weightstats import zconfint

In [4]:
np.array([49,58,75,110,112,132,151,276,281,362]) - 200

array([-151, -142, -125,  -90,  -88,  -68,  -49,   76,   81,  162])

In [5]:
stats.wilcoxon(np.array([49,58,75,110,112,132,151,276,281,362]) - 200)

WilcoxonResult(statistic=17.0, pvalue=0.2845026979112075)

In [6]:
stats.mannwhitneyu([17,18,18,15,12,4,14,15,10], [22,22,15,13,19,19,18,20,21,13,13,15])

MannwhitneyuResult(statistic=27.0, pvalue=0.02900499272087373)

In [7]:
df = pd.read_csv('challenger.txt', sep='\t')
df.head()

,Unnamed: 0,Temperature,Incident
0,Apr12.81,18.9,0
1,Nov12.81,21.1,1
2,Mar22.82,20.6,0
3,Nov11.82,20.0,0
4,Apr04.83,19.4,0


In [8]:
bad = df[df['Incident'] == 1]['Temperature'].to_numpy()
good = df[df['Incident'] == 0]['Temperature'].to_numpy()
bad.shape, good.shape

((7,), (16,))

In [9]:
def permutation_t_stat_ind(sample1, sample2):
    return np.mean(sample1) - np.mean(sample2)

def get_random_combinations(n1, n2, max_combinations):
    index = list(range(n1 + n2))
    indices = set([tuple(index)])
    for i in range(max_combinations - 1):
        np.random.shuffle(index)
        indices.add(tuple(index))
    return [(index[:n1], index[n1:]) for index in indices]

def permutation_zero_dist_ind(sample1, sample2, max_combinations = None):
    joined_sample = np.hstack((sample1, sample2))
    n1 = len(sample1)
    n = len(joined_sample)
    
    if max_combinations:
        indices = get_random_combinations(n1, len(sample2), max_combinations)
    else:
        indices = [(list(index), filter(lambda i: i not in index, range(n))) \
                    for index in itertools.combinations(range(n), n1)]
    
    distr = [joined_sample[list(i[0])].mean() - joined_sample[list(i[1])].mean() \
             for i in indices]
    return distr

def permutation_test(sample, mean, max_permutations = None, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    t_stat = permutation_t_stat_ind(sample, mean)
    
    zero_distr = permutation_zero_dist_ind(sample, mean, max_permutations)
    
    if alternative == 'two-sided':
        return sum([1. if abs(x) >= abs(t_stat) else 0. for x in zero_distr]) / len(zero_distr)
    
    if alternative == 'less':
        return sum([1. if x <= t_stat else 0. for x in zero_distr]) / len(zero_distr)

    if alternative == 'greater':
        return sum([1. if x >= t_stat else 0. for x in zero_distr]) / len(zero_distr)

In [11]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples
def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [15]:
np.random.seed(0)
good_bootstraped = get_bootstrap_samples(good, 1000)
bad_bootstraped = get_bootstrap_samples(bad, 1000)
stat_intervals(good_bootstraped.mean(axis=1) - bad_bootstraped.mean(axis=1), 0.05)

array([1.42299107, 7.93861607])

In [22]:
np.random.seed(0)
permutation_test(good, bad, max_permutations=10000)

0.007